<a id="1"></a> <br>
## Step 1 : Reading and Understanding Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt 
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [2]:
# Source:

# Dr Daqing Chen, Director: Public Analytics group. chend '@' lsbu.ac.uk, School of Engineering, London South Bank University, London SE1 0AA, UK.


# Data Set Information:

# This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.


# Attribute Information:

# InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation.
# StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product.
# Description: Product (item) name. Nominal.
# Quantity: The quantities of each product (item) per transaction. Numeric.
# InvoiceDate: Invoice Date and time. Numeric, the day and time when each transaction was generated.
# UnitPrice: Unit price. Numeric, Product price per unit in sterling.
# CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer.
# Country: Country name. Nominal, the name of the country where each customer resides.

In [4]:
retail = pd.read_csv(r"C:\\Users\\souri\Downloads\\OnlineRetail.csv"\OnlineRetail.csv", encoding= 'unicode_escape')
retail.head(10)

SyntaxError: unexpected character after line continuation character (Temp/ipykernel_13076/686585073.py, line 1)

In [ ]:
# shape of df

retail.shape

In [ ]:
# df info

retail.info()

In [ ]:
# df description

retail.describe()

<a id="2"></a> <br>
## Step 2 : Data Cleansing

In [ ]:
#for handing missing values:
#numerical-- fill the missing value by mean or median
#categorical data-- fill the missing value by mode of the column

In [ ]:
retail.isnull().sum()

In [ ]:
# Calculating the Missing Values % contribution in DF

df_null = round(100*(retail.isnull().sum())/len(retail), 2)
df_null

In [ ]:
#retail.dropna(inplace=True)

In [ ]:
# Droping rows having missing values

retail = retail.dropna()
retail.shape

In [ ]:
retail.isnull().sum()

In [ ]:
retail.dtypes

In [ ]:
retail.head(6)

In [ ]:
retail.loc[:,'CustomerID']=retail.loc[:,'CustomerID'].astype(str)

In [ ]:
# Changing the datatype of Customer Id as per Business understanding

retail['CustomerID'] = retail['CustomerID'].astype(str)

In [ ]:
retail.dtypes

<a id="3"></a> <br>
## Step 3 : Data Preparation

# We are going to analysis the Customers based on below 3 factors:
- R (Recency): Number of days since last purchase
- F (Frequency): Number of tracsactions
- M (Monetary): Total amount of transactions (revenue contributed)

In [ ]:
retail.head()

In [ ]:
# New Attribute : Monetary

retail['Amount'] = retail['Quantity']*retail['UnitPrice']


In [ ]:
retail.head(10)

In [ ]:
rfm_m = retail.groupby('CustomerID')['Amount'].sum()
rfm_m.head()

In [ ]:
rfm_m

In [ ]:
rfm_m = rfm_m.reset_index()
rfm_m.head()

In [ ]:
retail.head()

In [ ]:
# New Attribute : Frequency

rfm_f = retail.groupby('CustomerID')['InvoiceNo'].count()

In [ ]:
rfm_f

In [ ]:
rfm_f=rfm_f.reset_index()

In [ ]:
rfm_f.head()

In [ ]:
rfm_f.columns

In [ ]:
#rfm_f = rfm_f.reset_index()
rfm_f.columns = ['CustomerID', 'Frequency']
rfm_f.head()

In [ ]:
rfm_m

In [ ]:
# Merging the two dfs
rfm = pd.merge(rfm_m, rfm_f, on='CustomerID', how='inner')
rfm.head()

In [ ]:
rfm.shape

In [ ]:
retail.head()

In [ ]:
retail.dtypes

In [ ]:
# New Attribute : Recency

# Convert to datetime to proper datatype

retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'],format='%m/%d/%Y %H:%M')


In [ ]:
retail.head()

In [ ]:
retail.dtypes

In [ ]:
#apple-- 20th jan-- 10days-- 202
#pencil-- 21st jan-- 9 days-- 201
#products-- 3rd jan-- 27 days-- 202
#any_item--- 30th jan-- 0 day-- 209


In [ ]:
#10 day back-- 10 jan
#7 days back-- 17 jan-30th jan-- 

#20 day back-- 20 jan
#3 day back-- 23 jan



In [ ]:
# Compute the maximum date to know the last transaction date

max_date = max(retail['InvoiceDate'])
max_date

In [ ]:
# Compute the difference between max date and transaction date

retail['Diff'] = max_date - retail['InvoiceDate']
retail.head()

In [ ]:
# Compute last transaction date to get the recency of customers

rfm_p = retail.groupby('CustomerID')['Diff'].min()
rfm_p = rfm_p.reset_index()
rfm_p.head()

In [ ]:
rfm_p.shape

In [ ]:
retail.tail()

In [ ]:
rfm_p

In [ ]:
# Extract number of days only

rfm_p['Diff'] = rfm_p['Diff'].dt.days
rfm_p.head()

In [ ]:
#dir(dt)

In [ ]:
rfm.head()

In [ ]:
rfm_p.head()

In [ ]:
rfm_p.dtypes

In [ ]:
#rfm_p['CustomerID'] = rfm_p.CustomerID.astype(str)

In [ ]:
rfm=rfm.loc[:,['CustomerID','Amount','Frequency']]

In [ ]:
rfm

In [ ]:
rfm_p

In [ ]:
# Merge tha dataframes to get the final RFM dataframe

rfm = pd.merge(rfm, rfm_p, on='CustomerID', how='inner')


In [ ]:
rfm

In [ ]:
rfm.columns = ['CustomerID', 'Amount', 'Frequency', 'Recency']
rfm.head()

In [ ]:
rfm.describe()

In [ ]:
# Outlier Analysis of Amount Frequency and Recency

attributes = ['Amount','Frequency','Recency']
#plt.rcParams['figure.figsize'] = [14,8]
plt.figure(figsize=(9,7))

sns.boxplot(data = rfm[attributes])

plt.title("Outliers Variable Distribution", fontsize = 14, fontweight = 'bold')
plt.ylabel("Range", fontweight = 'bold')
plt.xlabel("Attributes", fontweight = 'bold')

In [ ]:
#2 pm iST or 3pm iST

In [ ]:
Q1 = rfm.Amount.quantile(0.25)
Q3 = rfm.Amount.quantile(0.75)
IQR = Q3 - Q1

In [ ]:
IQR

In [ ]:
Q1

In [ ]:
Q3

In [ ]:
Q1 - 1.5*IQR

In [ ]:
Q3 + 1.5*IQR

In [ ]:
# Removing (statistical) outliers for Amount
Q1 = rfm.Amount.quantile(0.25)
Q3 = rfm.Amount.quantile(0.75)
IQR = Q3 - Q1
rfm = rfm[(rfm.Amount >= Q1 - 1.5*IQR) & (rfm.Amount <= Q3 + 1.5*IQR)]

# Removing (statistical) outliers for Recency
Q1 = rfm.Recency.quantile(0.25)
Q3 = rfm.Recency.quantile(0.75)
IQR = Q3 - Q1
rfm = rfm[(rfm.Recency >= Q1 - 1.5*IQR) & (rfm.Recency <= Q3 + 1.5*IQR)]

# Removing (statistical) outliers for Frequency
Q1 = rfm.Frequency.quantile(0.25)
Q3 = rfm.Frequency.quantile(0.75)
IQR = Q3 - Q1
rfm = rfm[(rfm.Frequency >= Q1 - 1.5*IQR) & (rfm.Frequency <= Q3 + 1.5*IQR)]

In [ ]:
rfm

In [ ]:
rfm.describe()

In [ ]:
rfm

In [ ]:
# Rescaling the attributes

rfm_df = rfm[['Amount', 'Frequency', 'Recency']]

# Instantiate
scaler = StandardScaler()

# fit_transform
rfm_df_scaled = scaler.fit_transform(rfm_df)#mean= 0 and sd=1
rfm_df_scaled

In [ ]:
rfm_df_scaled

In [ ]:
rfm_df_scaled = pd.DataFrame(rfm_df_scaled)
rfm_df_scaled.columns = ['Amount', 'Frequency', 'Recency']
rfm_df_scaled.head()

In [ ]:
rfm_df_scaled

<a id="4"></a> <br>
## Step 4 : Building the Model

### K-Means Clustering

In [ ]:
# k-means with some arbitrary k

kmeans = KMeans(n_clusters=4)
kmeans.fit(rfm_df_scaled)

In [ ]:
kmeans.labels_

### Finding the Optimal Number of Clusters

#### Elbow Curve to get the right number of Clusters
A fundamental step for any unsupervised algorithm is to determine the optimal number of clusters into which the data may be clustered. The Elbow Method is one of the most popular methods to determine this optimal value of k.

In [ ]:
#for the 1st loop, num_cluster=2
#it will create a kmeans model for 2 num of clster
#feeding the scaled data in 2 cluster k means model 

In [ ]:
# Elbow-curve/SSD
# inertia
# Sum of squared distances of samples to their closest cluster center.

ssd = [] #store the ssd data
range_n_clusters = [2, 3, 4, 5, 6, 7, 8]# number of clusters
for num_clusters in range_n_clusters:
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(rfm_df_scaled)
    ssd.append(kmeans.inertia_)
ssd

In [ ]:
# plot the SSDs for each n_clusters
x=[2,3,4,5,6,7,8]
y=ssd
plt.plot(x,y)

In [ ]:
# Final model with k=5
kmeans = KMeans(n_clusters=5)
kmeans.fit(rfm_df_scaled)

In [ ]:
 kmeans.labels_

In [ ]:
rfm_df_scaled

In [ ]:
rfm

In [ ]:
# assign the label
rfm['Cluster_Id'] = kmeans.labels_
rfm.head()

In [ ]:
# Box plot to visualize Cluster Id vs Frequency

sns.boxplot(x='Cluster_Id', y='Amount', data=rfm)

In [ ]:
# Box plot to visualize Cluster Id vs Frequency

sns.boxplot(x='Cluster_Id', y='Frequency', data=rfm)

In [ ]:
# Box plot to visualize Cluster Id vs Recency

sns.boxplot(x='Cluster_Id', y='Recency', data=rfm)

In [ ]:
plt.scatter(rfm['Amount'],rfm['Cluster_Id'], m)

In [ ]:
rfm

In [ ]:
rfm[rfm['Cluster_Id']==2]

In [ ]:
rfm[rfm['Cluster_Id']==4]

### Inference:
K-Means Clustering with 3 Cluster Ids
- Customers with Cluster Id 2 are the customers with high amount of transactions as compared to other customers.
- Customers with Cluster Id 2 are frequent buyers.
- Customers with Cluster Id 0 are not recent buyers and hence least of importance from business point of view.